In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from sklearn.model_selection import train_test_split

In [2]:
#csvファイルの読み込み
race_results = pd.read_csv('race_results_2020.csv',encoding="shift jis")

In [9]:
#4着以下をすべて4にする
clip_rank = lambda x: x if x < 4 else 4
race_results["rank"] = race_results["着順"].map(clip_rank)

#カテゴリ変数をダミー変数化
race_results_d = pd.get_dummies(race_results)

#訓練データとテストデータに分ける
from sklearn.model_selection import train_test_split

X = race_results_d.drop(["rank"], axis=1)
y = race_results_d["rank"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=0
)

#アンダーサンプリング
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(
    sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1},
    random_state=71
)

X_train_rus, y_train_rus = rus.fit_resample(X_train.values, y_train.values)

#訓練
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

#スコアを表示
print(model.score(X_train, y_train), model.score(X_test, y_test))

#予測結果を確認
y_pred = model.predict(X_test)
pred_df = pd.DataFrame({"pred": y_pred, "actual": y_test})
pred_df[pred_df["pred"] == 1]["actual"].value_counts()


#回帰係数の確認
coefs = pd.Series(model.coef_[0], index=X.columns).sort_values()
coefs[["枠番", "馬番", "斤量", "単勝", "人気", "年齢", "体重", "体重変化"]]

0.7832482917076955 0.7832625496066281


枠番     -3.158104e-25
馬番     -4.959666e-24
斤量      2.935969e-24
単勝     -9.754584e-22
人気     -9.409583e-23
年齢     -3.491366e-24
体重      1.427690e-22
体重変化    1.170688e-23
dtype: float64

In [7]:
#現在のdfを確認
race_results

,race_id,着順,枠番,馬番,馬名,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化,rank
0,202001010101,1,6,6,ウインルーア,54.0,横山武史,1:49.7,35.6,16.0,3,牝,2,438,4,1
1,202001010101,2,2,2,アークライト,54.0,ルメール,1:50.0,35.8,1.9,2,牡,2,510,0,2
2,202001010101,3,3,3,ギャラントウォリア,54.0,池添謙一,1:50.1,36.2,1.8,1,牡,2,482,-6,3
3,202001010101,4,1,1,ジュンブーケ,52.0,亀田温心,1:50.5,36.7,22.2,4,牝,2,442,0,4
4,202001010101,5,4,4,キタノマンゲツ,54.0,藤岡康太,1:51.0,36.6,55.7,5,牡,2,426,-8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47871,202010020812,11,1,1,キスラー,57.0,武豊,1:45.5,37.7,6.5,4,牡,4,476,-2,4
47872,202010020812,12,8,14,ポンペイワーム,57.0,浜中俊,1:45.6,38.0,19.8,8,牡,4,522,-8,4
47873,202010020812,13,8,15,アルマトップエンド,57.0,国分恭介,1:45.9,37.3,75.0,14,牡,5,510,0,4
47874,202010020812,14,2,3,ノイーヴァ,55.0,太宰啓介,1:46.7,37.8,46.1,10,牝,4,488,-12,4


In [4]:
#dfのデータ型を知りたい場合
#race_results.info()

#データ型を変換
#race_results['タイム'] = pd.to_datetime(race_results['タイム'], format='%M:%S.%f').dt.time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47876 entries, 0 to 47875
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   race_id  47876 non-null  int64  
 1   着順       47876 non-null  int64  
 2   枠番       47876 non-null  int64  
 3   馬番       47876 non-null  int64  
 4   馬名       47876 non-null  object 
 5   斤量       47876 non-null  float64
 6   騎手       47876 non-null  object 
 7   タイム      47876 non-null  object 
 8   上り       47876 non-null  float64
 9   単勝       47876 non-null  float64
 10  人気       47876 non-null  int64  
 11  性        47876 non-null  object 
 12  年齢       47876 non-null  int64  
 13  体重       47876 non-null  int64  
 14  体重変化     47876 non-null  int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 5.5+ MB
